<a href="https://colab.research.google.com/github/inverter404/bosch-interiit/blob/main/train_k_600_GrayBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Installing required Libraries**

In [ ]:
!pip install -q --upgrade --no-cache-dir gdown
!pip install scikit-video
import gdown

**Importing Libraries**

In [ ]:
import os
import torch
import torch.nn.parallel
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import *
import torchvision
import torch.nn as nn
import skvideo.io
import cv2
import numpy as np
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from tqdm import tqdm
from torchsummary import summary

In [ ]:
!rm -rf /content/pre-trained/
os.mkdir('/content/pre-trained')
!cd '/content/pre-trained/'
!gdown --id 1d9KUOPsmOX_Z9YsxPZ96Juee8dBs-qT5

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1d9KUOPsmOX_Z9YsxPZ96Juee8dBs-qT5
To: /content/best.pth.tar
100% 176M/176M [00:01<00:00, 93.9MB/s]


In [ ]:
!git clone https://github.com/MCG-NJU/TDN

fatal: destination path 'TDN' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.append('/content/TDN/')

In [ ]:
from ops.models import TSN

**Loading Model**

In [ ]:
model=torch.load("/content/best.pth.tar")

**Defining Thief Model**

In [ ]:
thief_model = TSN(174,3,'RGB',
                base_model='resnet101',
                consensus_type='avg',
                dropout=0.5,
                img_feature_dim=256,
                partial_bn=False,
                pretrain=False,
                fc_lr5=False)


    Initializing TSN with base model: resnet101.
    TSN Configurations:
        input_modality:     RGB
        num_segments:       3
        new_length:         1
        consensus_module:   avg
        dropout_ratio:      0.5
        img_feature_dim:    256
            
=> base model: resnet101


In [ ]:
model=model['state_dict']

In [ ]:
state={}

In [ ]:
for (key,value) in model.items():
  state[key[7:]]=model[key]

In [ ]:
thief_model.load_state_dict(state)

<All keys matched successfully>

In [ ]:
thief_model

TSN(
  (base_model): TDN_Net(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv1_temp): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (conv1_5): Sequential(
      (0): Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (maxpool_diff): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (resnext_layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(64, eps=1e-05, momen

In [ ]:
thief_model.new_fc=nn.Linear(in_features=2048, out_features=600, bias=True) # defining last layer as kinetics 400 dataset has 400 classes

**Storing Video path for training**

In [ ]:
video_path_train=[]
labels_train=[]
video_path_val=[]
labels_val=[]
for i in tqdm(range(600)):
  dir=f'/content/drive/MyDrive/Kinetics 600/train/class-{i+1}'
  for j,name in enumerate(os.listdir(dir)):
    name=dir+str('/')+name
    video_path_train.append(name)
    labels_train.append(i)
video_path_train=np.array(video_path_train)
labels_train=np.array(labels_train)

100%|██████████| 600/600 [00:00<00:00, 1112.01it/s]


In [ ]:
video_path_train

array(['/content/drive/MyDrive/Kinetics 600/train/class-1/0pP5PNGEDi0_000161.mp4',
       '/content/drive/MyDrive/Kinetics 600/train/class-1/0s2mvbfzzyc_000019.mp4',
       '/content/drive/MyDrive/Kinetics 600/train/class-1/1oG6YwxGfCc_000038.mp4',
       ...,
       '/content/drive/MyDrive/Kinetics 600/train/class-600/2g_5Wzo1mJM_000022.mp4',
       '/content/drive/MyDrive/Kinetics 600/train/class-600/2J0SWxs5sgk_000042.mp4',
       '/content/drive/MyDrive/Kinetics 600/train/class-600/4TI-BUYEnIU_000008.mp4'],
      dtype='<U74')

**Creating dataset**

In [ ]:
class Vid_Dataset(Dataset):
    '''
    creates dataset of videos from k400 folder
    16 frames are extracted from each video
    '''
    def __init__(self,video_path,labels):
        self.n_samples=labels.shape[0]
        self.video_path=video_path
        self.labels=labels
    def __getitem__(self, index):
        vid=cv2.VideoCapture(self.video_path[index])
        array=[]
        success = 1
        count=0
        cnt=0
        length=(int(vid.get(cv2.CAP_PROP_FRAME_COUNT)))//16
        while success:
            success, image = vid.read()
            count += 1
            if(count%length==0 and cnt<16):
                cnt+=1
                array.append(np.array(image))
        array=np.array(array)
        array=array.astype('float32')
        sample=(array,self.labels[index])
        return sample
    def __len__(self):
        return self.n_samples

In [ ]:
for params in thief_model.parameters():
  params.requires_grad=False
for params in thief_model.new_fc.parameters():
  params.requires_grad=True

In [ ]:
dataset1=Vid_Dataset(video_path_train,labels_train)

**DataLoader**

In [ ]:
train_loader=DataLoader(dataset=dataset1,batch_size=1,shuffle=True)

In [ ]:
device=torch.device('cuda:0')

In [ ]:
theif_model=thief_model.to(device)

**Loss, Optimizer and Scheduler**

In [ ]:
criteria= nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(thief_model.parameters(),lr=1e-3,momentum=0.9)
decay_scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)


**Test loop**

In [ ]:
def test(model):
  thief_model.eval()
  with torch.no_grad():
    cnt=0
    num=0
    for inputs,labels in (val_loader):
      try:
        inputs=np.squeeze(inputs)
        inputs=inputs.permute(3,0,1,2)
        inputs=inputs.to(device)
  
        labels=labels.to(device)
        out=model(inputs,no_reshape=True)
        _,pred=torch.max(out,axis=1)
        if(int(pred.item())==labels):
          cnt+=1
        num+=1
      except:
        continue
      print(f'Validation Accuracy: {(cnt/num)*100}')

**Train Loop**

In [ ]:
def train(model,criteria,optimizer,decay_scheduler,num_epochs):
  thief_model.train()
  for i in range(num_epochs):
    epoch_loss=0
    num=0
    print(f"XXXXXXXXXXXXXXXXXX EPOCH-{i} XXXXXXXXXXXXXXXXXXX")
    for inputs,labels in (train_loader):
      inputs=np.squeeze(inputs)
      inputs=inputs.permute(3,0,1,2)
      inputs=inputs.to(device)
      labels=labels.to(device)
      out=model(inputs,no_reshape=True)
      loss=criteria(out,labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if(num%20==0):
        print(loss.item())
      num+=1
      epoch_loss+=loss.item()
      
    state={
        'model':model,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state,f'/content/checkpoints/check-{i}')
    test(model)
    print(epoch_loss/num)
    scheduler.step(epoch_loss)

  return model

**Training**

In [ ]:
thief=train(thief_model,criteria,optimizer,decay_scheduler,20)

XXXXXXXXXXXXXXXXXX EPOCH-0 XXXXXXXXXXXXXXXXXXX
6.1708269119262695


RuntimeError: ignored